<a href="https://colab.research.google.com/github/javier-marti-isasi/03MAIR-Algoritmos-de-optimizacion/blob/main/Javier_Marti_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Algoritmos - Actividad guiada 3
Nombre: Javier Martí<br>
URL: https://colab.research.google.com/drive/11s9cQI7a5xIiHHHniHnRwqt7LNWWNloz?usp=sharing

https://github.com/javier-marti-isasi/03MAIR-Algoritmos-de-optimizacion

In [ ]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

In [3]:
import tsplib95       #Modulo para las instancias del problema del TSP
import random         #Modulo para generar números aleatorios
from math import e    #constante e
import copy           #Para copia profunda de estructuras de datos(en python la asignación es por referencia)
import math

In [ ]:
import urllib.request #Hacer llamadas http a paginas de la red

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file) 

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/eil51.tsp", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/att48.tsp", file)


In [ ]:
#Modulos extras, no esenciales
import numpy as np
import matplotlib.pyplot as plt
import imageio                    #Para construir las imagenes con gif
from google.colab import files    #Para descargar ficheros generados con google colab

from tempfile import mkstemp      #Para genera carpetas y ficheros temporales
#import tempfile

In [5]:
#Carga de datos y generación de objeto problem
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

In [8]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
#dir(problem)

15

In [9]:
#Funcionas basicas      
     
#Se genera una solucion aleatoria con comienzo en en el nodo 0 (por convención)
#solucion=[0,1,2,3,4......,41]
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion 

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [18]:
solucion = crear_solucion(Nodos)
print(solucion)
distancia_total(solucion, problem)

[0, 36, 13, 15, 9, 19, 24, 41, 40, 18, 5, 16, 33, 10, 21, 8, 23, 11, 3, 7, 37, 28, 22, 25, 29, 27, 2, 6, 31, 17, 26, 39, 38, 1, 34, 14, 12, 35, 4, 32, 30, 20]


4335

In [57]:
def busqueda_aleatoria(problem, N):
  
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  mejor_distancia = 10e100                          #Inicializamos con un valor alto
  
  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)
    
    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia
      
      
  print("Mejor solución:" , mejor_solucion) 
  print("Distancia     :" , mejor_distancia) 
  return mejor_solucion  
    

#Creamos 5000 soluciones y nos quedamos con la mejor
solucion = busqueda_aleatoria(problem, 5000)    

Mejor solución: [0, 21, 41, 22, 34, 13, 19, 6, 40, 39, 25, 29, 2, 3, 15, 16, 1, 33, 37, 17, 36, 31, 7, 4, 32, 30, 27, 28, 18, 5, 11, 14, 24, 8, 10, 9, 38, 26, 12, 23, 20, 35]
Distancia     : 3616


In [59]:
def genera_vecina(solucion):
  #Calculamos una única vecindad, un solo ciclo
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):
      
      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]  

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)         

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [0, 21, 41, 22, 34, 13, 19, 6, 40, 39, 25, 29, 2, 3, 15, 16, 1, 33, 37, 17, 36, 31, 7, 4, 32, 30, 27, 28, 18, 5, 11, 14, 24, 8, 10, 9, 38, 26, 12, 23, 20, 35]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))

nueva_solucion = genera_vecina(solucion)
print("Distancia Solucion Local:", distancia_total(nueva_solucion, problem))
print(nueva_solucion)

Distancia Solucion Incial: 3616
Distancia Solucion Local: 3387
[0, 21, 41, 22, 34, 13, 19, 6, 40, 39, 25, 29, 2, 3, 15, 16, 1, 33, 37, 17, 36, 31, 7, 4, 32, 30, 27, 28, 18, 5, 11, 23, 24, 8, 10, 9, 38, 26, 12, 14, 20, 35]


In [70]:
#De forma recurrente generamos vecinas, avalúo la funcion de coste (distancia) y si es mejor que la mejor vecina anterior encontrada, la reemplazamos
#Primero hemos empezado con un componente aleatorio y, a partir de ella, ahora hacemos una búsqueda local de manera determinista
#Paramos cuando todas las vecinas no superan la mejor distancia, llegamos a un valle

#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []
  
  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0   #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1     #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)
    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)
 
    #Si no mejoramos hay que terminas. Hemos llegado a un minimo local(según nuestro operador de vencindad)
    if distancia_vecina < mejor_distancia:
      mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia 
      mejor_distancia = distancia_vecina
    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion) 
      print("Distancia     :" , mejor_distancia) 
      return mejor_solucion
      
    solucion_referencia = vecina
 

sol = busqueda_local(problem )  

En la iteracion  26 , la mejor solución encontrada es: [0, 1, 19, 13, 5, 6, 3, 32, 20, 34, 30, 9, 23, 41, 25, 11, 12, 18, 26, 4, 2, 27, 14, 16, 15, 36, 35, 33, 38, 21, 24, 40, 29, 28, 10, 8, 39, 22, 7, 37, 17, 31]
Distancia     : 1993


In [ ]:
#Generador de 1 solucion vecina 2-opt aleatoria (intercambiar 2 nodos)
#MEJORABLE!!! POr eso el resultado con Recocido Simulado no son muy mejorable
# No tendría que ser tan aleatorio pero 

def genera_vecina_aleatorio(solucion):

  #Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  
  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  
genera_vecina_aleatorio(solucion) 

In [73]:
#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99

In [78]:
def recocido_simulado(problem, TEMPERATURA ):
  #problem = datos del problema
  #T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
    
  N=0
  while TEMPERATURA > .0001:
  #Vamos descendiendo la temperatura
    N+=1
    #Genera una solución vecina
    vecina =genera_vecina_aleatorio(solucion_referencia)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
      
    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia  
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina), cambiamos la referencia
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      solucion_referencia = copy.deepcopy(vecina)
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)
 
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol  = recocido_simulado(problem, 10000000)


La mejor solución encontrada es [0, 6, 4, 3, 32, 28, 2, 27, 1, 37, 31, 20, 34, 38, 22, 9, 8, 33, 35, 36, 17, 15, 7, 18, 12, 29, 30, 39, 21, 24, 40, 23, 41, 10, 25, 11, 5, 16, 14, 19, 13, 26]
con una distancia total de 1986


 1273, mejor solución encontrada